In [1]:
import pandas as pd
import os
import subprocess
import numpy as np
import random

# Creating the reference.smap from the igsr_samples.tsv

In [3]:
#since the same sample are in the three different file we can choose any file we want here
reference_file='../data/input/real/65k_SNP/65kSNP_chr1.vcf.gz'

legend_file='../data/input/real/igsr_samples.tsv'
reference_map_file='../data/input/real/reference.smap'

#creating the .smap file
sample_list=pd.read_csv(legend_file,sep="\t")
reference_map= sample_list.loc[:, ['Sample name', 'Superpopulation code']]

#deleting Frow if superpopulation code is Nan
reference_map = reference_map.dropna(subset=['Superpopulation code'])

#removing the EUR,AFR individual
reference_map=reference_map[reference_map.iloc[:,1]!='EUR,AFR']

sample_name = subprocess.check_output(['bcftools', 'query', '-l', reference_file], universal_newlines=True)
sample_name=list(sample_name.strip().split('\n'))

#deleting row if the sample is not in the reference file
mask= reference_map['Sample name'].isin(sample_name)
reference_map = reference_map[mask]

np.savetxt(reference_map_file,reference_map,delimiter='\t',fmt="%s")

## Extracting specific subpopulations from igsr file

In this example we want to extract PUR subpopulation from the referece vcf file.

In [68]:
legend_file='./data/input/real/igsr_samples.tsv'
df=pd.read_csv(legend_file,sep='\t')
reference_map_file='./data/input/real/65k_SNP/reference.smap'
reference_map=pd.read_csv(reference_map_file,delimiter='\t',header=None)
reference_map.columns=["Sample name","Population code"]
reference_map

,Sample name,Population code
0,HG00271,EUR
1,HG00276,EUR
2,HG00288,EUR
3,HG00290,EUR
4,HG00308,EUR
...,...,...
2498,NA21143,SAS
2499,NA21112,SAS
2500,NA21117,SAS
2501,NA21124,SAS


In [69]:
df

,Sample name,Sex,Biosample ID,Population code,Population name,Superpopulation code,Superpopulation name,Population elastic ID,Data collections
0,HG00271,male,SAME123417,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
1,HG00276,female,SAME123424,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
2,HG00288,female,SAME1839246,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
3,HG00290,male,SAME1839057,FIN,Finnish,EUR,European Ancestry,FIN,"1000 Genomes on GRCh38,1000 Genomes 30x on GRC..."
4,HG00303,male,SAME1840115,FIN,Finnish,EUR,European Ancestry,FIN,1000 Genomes on GRCh38
...,...,...,...,...,...,...,...,...,...
4973,HGDP00773,female,SAMEA3302906,NaN,Japanese,NaN,East Asia (SGDP),JapaneseSGDP,Simons Genome Diversity Project
4974,R3,male,SAMEA3302714,NaN,Relli,NaN,South Asia (SGDP),RelliSGDP,Simons Genome Diversity Project
4975,NA12236,female,SAMEA6604124,CEU,CEPH,EUR,European Ancestry,CEU,1000 Genomes 30x on GRCh38
4976,HGDP00456,male,NaN,NaN,Mbuti,NaN,Africa (HGDP),MbutiHGDP,HGDP Transcriptome


In [71]:
PUR=df[df["Population code"]=='PUR']
sample_PUR=PUR["Sample name"]
print(reference_map)
print('\n')
print(sample_PUR)

sample_PUR=sample_PUR.to_frame().merge(reference_map)
sample_PUR

     Sample name Population code
0        HG00271             EUR
1        HG00276             EUR
2        HG00288             EUR
3        HG00290             EUR
4        HG00308             EUR
...          ...             ...
2498     NA21143             SAS
2499     NA21112             SAS
2500     NA21117             SAS
2501     NA21124             SAS
2502     NA21129             SAS

[2503 rows x 2 columns]


101     HG00555
109     HG00733
111     HG00738
113     HG00740
119     HG01161
         ...   
2769    HG01303
2772    HG01308
2811    HG01396
2814    HG01404
2817    HG01411
Name: Sample name, Length: 150, dtype: object


,Sample name,Population code
0,HG00740,AMR
1,HG01161,AMR
2,HG01173,AMR
3,HG01197,AMR
4,HG01200,AMR
...,...,...
99,HG01098,AMR
100,HG01101,AMR
101,HG01303,AMR
102,HG01308,AMR


In [72]:
np.savetxt("./data/input/real/PUR_sample.tsv", sample_PUR['Sample name'], delimiter="\t", fmt="%s")
np.savetxt("./data/input/real/PUR_sample.smap",sample_PUR,delimiter='\t',fmt='%s')